# COLLECTING IMDBIDS AND TITLES
---

Nhiệm vụ phần này: từ danh sách các đường dẫn phim của trang web `Just Watch`, thực hiện cào thông tin **imdbId** và **title** trong đường dẫn nếu không có id và lưu lại vào file ids_titles.txt.

## 1. IMPORT THƯ VIỆN CẦN THIẾT

In [2]:
import time
import os
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import re

## 2. CÀO THÔNG TIN IMDBID TRONG TRANG `JUST WATCH`

In [3]:
links = []

with open('../Data/links.txt', 'r') as f:
    links = [line.strip() for line in f]

Dùng selenium thoi vì request k được

In [4]:
# chia 3 ra nha mina hong no k cho minh lay :D
# từ từ để ngâm cú chứ nhìn này trông cx hơi ngu :v
member_part = {
    "Trinh": {
        "part": 1,
        "api_key": None, # đk api xong áp vô nè
        "start": 0,
        "end": 628
    },

    "Thi": {
        "part": 2,
        "api_key": None, # đk api xong áp vô nè
        "start": 808,
        "end": 1258
    },

    "MinhUyen": {
        "part": 3,
        "api_key": 'd4c83dec',
        "start": 1259,
        "end": 1887
    } 
}

In [5]:
# Điền tên mn vào đây theo từ điển nè
info = member_part["MinhUyen"]

part = info["part"]
apiKey = info["api_key"]
start = info["start"]
end = info["end"]

In [6]:
def collect_imdbID_title(links, start, end):
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

    imdbIds_titles = []
    for url in links[start: end]:
        print(f"[LOG]: parse imdbid of url {url}")
        driver.get(url)
        page_source = driver.page_source

        imdb_id_match = re.search(r'"imdbId":"(tt\d+)"', page_source)
        if imdb_id_match:
            imdb_id = imdb_id_match.group(1)
            imdbIds_titles.append(imdb_id)
            print("IMDB ID:", imdb_id)
        else:
            # print("Không tìm thấy IMDb ID.")
            movie_title_meta = driver.find_element(By.XPATH, "//meta[@property='og:title']")
            movie_title = movie_title_meta.get_attribute("content").split(" streaming")[0] if movie_title_meta else "Null"
            imdbIds_titles.append(movie_title)
            print(movie_title)

        time.sleep(3)

    driver.quit()
    return imdbIds_titles

In [ ]:
imdbIds_titles = collect_imdbID_title(links, start=start, end=end)
imdbIds_titles

[LOG]: parse imdbid of url https://www.justwatch.com/us/movie/the-mission-2023
IMDB ID: tt28097834
[LOG]: parse imdbid of url https://www.justwatch.com/us/movie/the-peasants
IMDB ID: tt10651230
[LOG]: parse imdbid of url https://www.justwatch.com/us/movie/the-burial-0
The Burial
[LOG]: parse imdbid of url https://www.justwatch.com/us/movie/saw-x
IMDB ID: tt21816322
[LOG]: parse imdbid of url https://www.justwatch.com/us/movie/last-call-for-istanbul
IMDB ID: tt23334616
[LOG]: parse imdbid of url https://www.justwatch.com/us/movie/coup-de-chance
IMDB ID: tt15140278
[LOG]: parse imdbid of url https://www.justwatch.com/us/movie/shooting-stars-2023
IMDB ID: tt3715152
[LOG]: parse imdbid of url https://www.justwatch.com/us/movie/wingwomen
IMDB ID: tt21638298
[LOG]: parse imdbid of url https://www.justwatch.com/us/movie/love-in-the-maldives
IMDB ID: tt26938504
[LOG]: parse imdbid of url https://www.justwatch.com/us/movie/one-day-as-a-lion
IMDB ID: tt15783426
[LOG]: parse imdbid of url https:/

## 3. LƯU THÔNG TIN VỀ ID VÀ TITLE VÀO FILE TXT

In [ ]:
file_name = os.path.join("./Data/", f"ids_title.csv")

with open(file_name, 'a') as f:
    for item in imdbIds_titles:
        f.write(item + "\n")